In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import twint
import nest_asyncio


%matplotlib inline

In [2]:
nest_asyncio.apply()

In [9]:
t = twint.Config()
t.Username = "YahooSportsNBA"
t.Since = '2021-01-31'
t.Pandas = True
twint.run.Search(t)

1371523338527444993 2021-03-15 14:07:00 -0400 <YahooSportsNBA> The @WNBA unveiled a new logo to celebrate its 25th anniversary 👀 #CountIt   https://t.co/vJwBnTPf7F
1371516350213029896 2021-03-15 13:39:14 -0400 <YahooSportsNBA> 🔘 JaVale McGee getting interest from the Nets 🔘 What's going on with DeMar DeRozan? 🔘 Nurk returning soon?   More notes from @ChrisBHaynes in Hoopla with Haynes ➡️  https://t.co/WyjsDGXVcE  https://t.co/r5yXvEatqR
1371286088405385216 2021-03-14 22:24:15 -0400 <YahooSportsNBA> ANOTHER DAY, ANOTHER ANTHONY EDWARDS JAM 💥   https://t.co/w96pJBWhUT
1371278872881336320 2021-03-14 21:55:35 -0400 <YahooSportsNBA> Jayson Tatum made this look too easy 👀   https://t.co/MyxA5l6b77
1371263992836943874 2021-03-14 20:56:27 -0400 <YahooSportsNBA> Jaylen faked 'em out 😂   https://t.co/MNMHPc0qFA
1371254793176375302 2021-03-14 20:19:54 -0400 <YahooSportsNBA> Ben Simmons with authority 🔨  (via @sixers)   https://t.co/CMlevDT3dv
1371242774528466945 2021-03-14 19:32:08 -0400 <YahooSp

1364892898911686658 2021-02-25 06:00:00 -0400 <YahooSportsNBA> "KD is the best player in the world right now."  @LaJethroJenkins and Seerat Sohi break down the red-hot Nets and which teams could potentially matchup best with them.
1364799134708867073 2021-02-24 23:47:25 -0400 <YahooSportsNBA> Spida too tough 😬  (via @utahjazz)   https://t.co/XNpGEU0ies
1364759455544406021 2021-02-24 21:09:44 -0400 <YahooSportsNBA> Myles says 🚫   https://t.co/6VciNRYd03
1364748956891234304 2021-02-24 20:28:01 -0400 <YahooSportsNBA> Anthony Edwards is sooo mean to the rims 😳   https://t.co/74enz4puOw
1364748302957219843 2021-02-24 20:25:25 -0400 <YahooSportsNBA> Gallo showing off the range 🎯   https://t.co/vPUvwO4Xtn
1364687975511183363 2021-02-24 16:25:42 -0400 <YahooSportsNBA> Fans logging on when they think their team got disrespected on national TV games...  https://t.co/9UwQjJ8A5Q
1364675723395403777 2021-02-24 15:37:01 -0400 <YahooSportsNBA> The NBA released the second half of the 2021 schedule.   

1359224066096443397 2021-02-09 14:34:05 -0400 <YahooSportsNBA> After 11 seasons, @ReneeMontgomery is retiring to focus on her social justice work in Atlanta.  ➡️  https://t.co/1Z63bp8bop  https://t.co/6pP4oAt3ob
1358988145140797443 2021-02-08 22:56:37 -0400 <YahooSportsNBA> Holy Murray 😱   https://t.co/8T4xggYYUO
1358973403214999568 2021-02-08 21:58:02 -0400 <YahooSportsNBA> Anthony Edwards got up 😤   https://t.co/97I4VqQukq
1358944748271140864 2021-02-08 20:04:10 -0400 <YahooSportsNBA> The typically old school Kevin Garnett was unexpectedly complimentary of today’s NBA 👀  ➡️  https://t.co/Tj2ZBB248Q  https://t.co/S25EEzg10v
1358895865566486531 2021-02-08 16:49:56 -0400 <YahooSportsNBA> "Racism is taught."  Donovan Mitchell and Rudy Gobert called out a Utah school for initially allowing parents to opt their students out of Black History Month curriculum.   ➡️  https://t.co/eaOm0nIwSg  https://t.co/et2OyFL6Pv
1358888071119450118 2021-02-08 16:18:57 -0400 <YahooSportsNBA> "You didn't los

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [10]:
df = twint.storage.panda.Tweets_df

In [11]:
df.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1371523338527444993,1371523338527444993,1.615832e+12,2021-03-15 14:07:00,-0400,,The @WNBA unveiled a new logo to celebrate its...,en,[countit],[],...,,,,,,[],,,,
1,1371516350213029896,1371516350213029896,1.615830e+12,2021-03-15 13:39:14,-0400,,🔘 JaVale McGee getting interest from the Nets ...,en,[],[],...,,,,,,[],,,,
2,1371286088405385216,1371286088405385216,1.615775e+12,2021-03-14 22:24:15,-0400,,"ANOTHER DAY, ANOTHER ANTHONY EDWARDS JAM 💥 h...",en,[],[],...,,,,,,[],,,,
3,1371278872881336320,1371278872881336320,1.615773e+12,2021-03-14 21:55:35,-0400,,Jayson Tatum made this look too easy 👀 https...,en,[],[],...,,,,,,[],,,,
4,1371263992836943874,1371263992836943874,1.615770e+12,2021-03-14 20:56:27,-0400,,Jaylen faked 'em out 😂 https://t.co/MNMHPc0qFA,en,[],[],...,,,,,,[],,,,


In [16]:
test_str = df.loc[4,'tweet']
encoded_test_str = test_str.encode('ascii', "ignore")
decoded_test_str = encoded_test_str.decode()
decoded_test_str

"Jaylen faked 'em out    https://t.co/MNMHPc0qFA"

In [3]:
def create_tweets_dataframe(username, start_date, end_date):
    c = twint.Config()
    c.Username = username
    c.Since = start_date
    c.Until = end_date
    c.Pandas = True
    c.Hide_output = True
    twint.run.Search(c)
    
    dataframe = twint.storage.panda.Tweets_df
    return dataframe
    
    
    

In [4]:
yahoo_df = create_tweets_dataframe('YahooSportsNBA', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [5]:
crossover_df = create_tweets_dataframe('TheCrossover', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [6]:
athletic_df = create_tweets_dataframe('TheAthleticNBA', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [7]:
slam_df = create_tweets_dataframe('Slamnewswire', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [9]:
nba_df = create_tweets_dataframe('NBA', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [10]:
bball_news_df = create_tweets_dataframe('basketbllnews', '2016-01-01', '2021-02-28')

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [11]:
len(yahoo_df) + len(crossover_df) + len(athletic_df) + len(slam_df) + len(nba_df) + len(bball_news_df)

185008

In [12]:
nba_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [13]:
df_list = [yahoo_df, crossover_df, athletic_df, slam_df, nba_df, bball_news_df]
for dataframe in df_list:
    dataframe.drop(columns=['id', 'conversation_id', 'created_at', 'timezone', 'place',
       'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'], inplace=True)

In [14]:
for dataframe in df_list:
    for date_string in dataframe['date']:
        dataframe['date'] = date_string[:10]

In [15]:
yahoo_df['source'] = 'Yahoo'
crossover_df['source'] = 'SI'
athletic_df['source'] = 'The Athletic'
slam_df['source'] = 'Slam'
nba_df['source'] = 'NBA'
bball_news_df['source'] = 'Basketball News'

In [16]:
twitter_df = pd.concat([yahoo_df, crossover_df, athletic_df, slam_df, nba_df, bball_news_df], ignore_index=True)

In [17]:
pd.set_option('display.max_colwidth', 100)
twitter_df.head()

,date,tweet,source
0,2021-02-26,"Obi Toppin is expected to participate in the NBA Slam Dunk Contest, per @ShamsCharania. 👀 https...",Yahoo
1,2021-02-26,MPJ SLAM 💥 https://t.co/bp6XsbTzUT,Yahoo
2,2021-02-26,Timberwolves G Malik Beasley has been suspended for 12 games due to conduct stemming from charge...,Yahoo
3,2021-02-26,Luka's game-winner against the Celtics vs. Luka's game-winner against the Clippers. Unreal 🤯 ...,Yahoo
4,2021-02-26,It's time to stop underrating the Utah Jazz 👀 From @KrystenPeek ⤵️,Yahoo


In [18]:
pd.set_option("display.max_rows", 101)
twitter_df.sample(n=100)

,date,tweet,source
92927,2021-02-26,At the half on @NBATV: @spurs 63 @LAClippers 54 DeMar DeRozan: 21 PTS LaMarcus Aldridge: 12 PT...,NBA
173968,2021-02-26,The @cavs hit Practice &amp; Media Avail on their home floor before Game 4 tomorrow on ABC. #NBA...,NBA
150400,2021-02-26,"Tonight the @Bucks host @Celtics at UW-Milwaukee Arena, formerly ""The MECCA"", 8pm/et @NBAonTNT! ...",NBA
48637,2021-02-26,"With Charlotte’s All-Star Weekend win, NBA endorses sham HB2 repeal (by @JATayler) https://t.co...",SI
169485,2021-02-26,James Harden posts his third triple double of the season in @HoustonRockets victory! #PhantomCam...,NBA
19691,2021-02-26,"NBA free agency winners and losers: The Nets front office, Celtics haters and more https://t.co...",SI
46438,2021-02-26,Chauncey Billups knew about Kyrie Irving's unhappiness before turning down Cavs job: https://t....,SI
130178,2021-02-26,🏆 x 🍾 x SWAG! #NBAFinals https://t.co/1lhKjbyHHD,NBA
20960,2021-02-26,Rich Paul reportedly complained to Adam Silver about Luke Walton early in the season https://t....,SI
45858,2021-02-26,LeBron is working with HBO on a comedy about a sneaker store and it's probably going to be dope....,SI


In [19]:
from bs4 import BeautifulSoup
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.cluster import KElbowVisualizer

In [20]:
def clean_tweets(text):
    """This function takes in a string (in this case a tweet) and cleans the string such that any and all
    non-ASCII charcaters and HTML characters references are removed.
    
    The string will still have all forms of punctuation.
    
    """
    text = re.sub(' +', ' ', text) #reduce instances of multiple blank spaces to a single space
    text = re.sub(r"http\S+", '', text) #remove links from tweets
    encoded_text = text.encode('ascii', 'ignore') #encode string using 'ascii' encoding
    decode_text = encoded_text.decode() #decode string (convert from bytes to string object)
    text_reclean = re.sub(' +', ' ', decode_text) #reduce instances of multiple blank spaces to a single space
    clean_text = BeautifulSoup(text_reclean, 'lxml').text #remove all html encodings such as &amp;, &quot;, etc.
    
    return clean_text

In [21]:
twitter_df['tweet'] = twitter_df['tweet'].apply(lambda x: clean_tweets(x))

In [22]:
def clean_tweets_modeling(text):
    """Performs a similar operation as the clean_tweets function except this function will create a list of 
    tokens that can be used for NLP modeling.
    
    For example, a tweet will be transformed such that all non-ASCII characters and HTML encodings are removed.
    Then the function will remove all punctuation from the tweet, split each word in the tweet by white space,
    and finally lemmatize each word in the tweet and group the lemmatized tokens in a list."""
    
    text = re.sub(' +', ' ', text) #reduce instances of multiple spaces to a single space
    encoded_text = text.encode('ascii', 'ignore') #encode text as ascii characters only
    decode_text = encoded_text.decode() #convert bytes back to a string
    clean_text = BeautifulSoup(decode_text, 'lxml').text #remove html tages using BeautifulSoup
    
    no_punc_text = "".join([word for word in clean_text if word not in string.punctuation]) #remove punctuation
    tokens = re.split('\W+', no_punc_text) #split words by in no_punc_text variable by whitespace
    
    #Lemmatize each token and store tokens in a list
    final_tokens = [wnl.lemmatize(word) for word in tokens if word not in stopwords]
    
    return final_tokens

In [23]:
twitter_df['tweet_tokens'] = twitter_df['tweet'].apply(lambda x: clean_tweets_modeling(x.lower()))

In [24]:
twitter_df.to_csv('nba_twitter.csv', index=False)

In [76]:
twitter_df.sample(n=100)

,date,tweet,source,tweet_tokens
45934,2021-02-26,Will Carmelo end up in Houston after all?,SI,"[carmelo, end, houston, ]"
178067,2021-02-26,Final minute. One. Point. Game. On the line: @NBAHistory. #ThisIsWhyWePlay,NBA,"[final, minute, one, point, game, line, nbahistory, thisiswhyweplay]"
132273,2021-02-26,CJ Miles is mic'd up for the @Raptors on #NBAonABC! #WeTheNorth #NBAPlayoffs,NBA,"[cj, mile, micd, raptor, nbaonabc, wethenorth, nbaplayoffs, ]"
13449,2021-02-26,"The looming, growing threat of Golden State in the pursuit of Kevin Durant. @WojVerticalNBA colu...",Yahoo,"[looming, growing, threat, golden, state, pursuit, kevin, durant, wojverticalnba, column, ]"
159776,2021-02-26,HALFTIME - #MileHighBasketball: 73 #DefendtheLand: 59 Nuggets are red hot with four players in d...,NBA,"[halftime, milehighbasketball, 73, defendtheland, 59, nugget, red, hot, four, player, doublefigu..."
78327,2021-02-26,"WHAT A GAME on NBA League Pass! @chicagobulls 114 @LAClippers 117 LaVine: 37 PTS, 9 3PM Kawhi: 3...",NBA,"[game, nba, league, pas, chicagobulls, 114, laclippers, 117, lavine, 37, pt, 9, 3pm, kawhi, 33, ..."
5303,2021-02-26,Roy Hibbert says the Night King will sit atop the Iron Throne at +1000 odds! Do you agree or dis...,Yahoo,"[roy, hibbert, say, night, king, sit, atop, iron, throne, 1000, odds, agree, disagree, roymeetsw..."
24280,2021-02-26,Stars like Zion Williamson may be barred from the NBA until they are a year removed from high sc...,SI,"[star, like, zion, williamson, may, barred, nba, year, removed, high, school, path, making, mone..."
64022,2021-02-26,Myles Turner's arrival could solve Pacers ongoing small-ball dilemma (by @RobMahoney),SI,"[myles, turner, arrival, could, solve, pacer, ongoing, smallball, dilemma, robmahoney, ]"
56482,2021-02-26,Great anecdote from Vic: He found out about the OKC trade on an airplane,SI,"[great, anecdote, vic, found, okc, trade, airplane, ]"


In [80]:
def tfidf_vectorizer(dataframe):
    
    tfidf = TfidfVectorizer(stop_words=stopwords)
    tfidf_sparse_mat = tfidf.fit_transform(dataframe['tweet'])
    
    tfidf_df = pd.DataFrame(tfidf_sparse_mat.toarray())
    tfidf_df.columns = tfidf.get_feature_names()
    
    return tfidf_df

In [81]:
twitter_tfidf_df = tfidf_vectorizer(twitter_df)

In [82]:
twitter_tfidf_df.shape

(185008, 33380)

In [ ]:
kmodel = KMeans()
tweet_viz = KElbowVisualizer(kmodel, k=(2,11), metric = 'calinski_harabasz', timings=False)
tweet_viz.fit(twitter_tfidf_df)
plt.title('Calinski-Harabasz Score for NBA Tweets Jan 2016 - Feb 2021')
plt.xlabel('Clusters')
plt.ylabel('CH Score')
plt.savefig('twitter_ch_plot.png')